In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweettxt/tweet.txt


Data Preprocessing:Pre-processing includes three primary steps. First step is Tokenization. It is the process of tokenizing each word in the sentence. Second step is Stemming and lemmatization. It is the process of achieving the root form of the derived words in the sentence, the words are further converted into present tense format. Third Step is The Removal of stop words. It is the process of removing the unnecessary words in the document, the commonly considered stop words include (the, a, an, in etc.,) these are some words that make no sense in the sentence.
Baseline: For our baselines, we have mentioned the scores acquired by the LSTM model. Utilised Semantic, lexicon-based aspects and sarcasm indicators to carry out two binary classification tasks: one for sarcastic label 1 and one for non-sarcastic label 0.
Trained a Logistic Regression and Recurrent Neural Network (with LSTM layer) methods to carry out classification of Ironic and Sarcastic Tweets using Natural Language Processing. Pre-processing of the text data and word vectorisation was required before training these methods using TensorFlow. Evaluat the methods' performance based on Precision, Recall, and F-Scores. The sequential method has produced accuracy over 60% for training and test datasets before impoved the Model.

Improved Model:
The above examination, the Keras model was overfitting. The model was performing reasonable on train data but was performing badly on test data. Calculating metric was also poor. The one of reasons for this poor performace was the pre-processing of data and the structure of the method. The preprocessing only changed string into lowercase and then generating tokens. The model had Stochastic Gradient Descent optimizer and did not have dropout layer. With imporoved model by adding Dropout layer as it randomly ignores the nodes and therefore avoides overfitting. By using the hing loss function in the model for better results because hinge loss works well for the binary classifcation.
Using Adaptive optimizer "adam". Preprocess the data and removing the stopwords, punctuations and special characters. To avoid any problem in tokenzing for the keras model Removing numbers from the data. The model performance was not improving very good but evaluation metricthe improved noticeably.

Achieved accuracy on the training dataset %99.


In [2]:
 #Required Imports
# %tensorflow_version 2.x
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from collections import Counter
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Embedding
from sklearn import svm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import itertools
from sklearn import metrics
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
print(tf.__version__)

2.6.4


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Loading data from csv file
data = pd.read_csv('../input/tweettxt/tweet.txt', sep='\t')
print(data.head())
# It is always better to check the null values in the dataset first.
print(data.isnull().any(axis = 0))

   Tweet index  Label                                         Tweet text
0            1      1  Sweet United Nations video. Just in time for C...
1            2      1  @mrdahl87 We are rumored to have talked to Erv...
2            3      1  Hey there! Nice to see you Minnesota/ND Winter...
3            4      0                3 episodes left I'm dying over here
4            5      1  I can't breathe! was chosen as the most notabl...
Tweet index    False
Label          False
Tweet text     False
dtype: bool


In [4]:
#loading the File
dataset = pd.read_csv('../input/tweettxt/tweet.txt', sep = '\t')

In [5]:
#Loding the file and storing in the dataframe
data = pd.read_csv('../input/tweettxt/tweet.txt', sep='\t')
#Renaming the columns
data.columns = ['Tweet_index', 'Label','Tweet_text']
#Storing the Tweet Text as list
corpusTweet = data['Tweet_text'].tolist() #Storing the lables as sentiments
sentiments = data['Label']
positiveExample = 0
negativeExample = 0
#Get all the words in one list
wordList =[]
for words in corpusTweet:
    wordList.append(words.lower().strip().split())
wordList = list(itertools.chain.from_iterable(wordList)) #Set is fetching the unique words and we add it to the vocabulary
vocabulary = set(wordList)
vocabulary_len = len(vocabulary)
for i in range(len(sentiments)):
    if(sentiments[i] == 1):
        positiveExample = positiveExample + 1
    else:
        negativeExample = negativeExample + 1
print("Size of the Vocabulary:",vocabulary_len)
print("Total Number of Positive Examples:",positiveExample)
print("Total Number of Negative Examples:",negativeExample)

Size of the Vocabulary: 17052
Total Number of Positive Examples: 1901
Total Number of Negative Examples: 1916


In [6]:
#Use the data for train test split
data_x = data['Tweet_text']
labels_y = data['Label']

In [7]:
#Function calculating precision recall and F-Measure
def calculate_metrics(y_true, y_pred):
    #Accept Actual values and Predicting values to calculate metric scores
    precision_score = metrics.precision_score(y_true, y_pred)
    recall_score = metrics.recall_score(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred)
    return precision_score, recall_score, f1_score

In [8]:
#Convert words into tokens(numbers)
vectorizer_word = CountVectorizer(ngram_range=(1,1))
vectorizer_word.fit(corpusTweet)
#transform output to array
data_x = vectorizer_word.transform(corpusTweet).toarray() #Split train test data
#splliting 70% for training and 30% for test
train_X, test_X, y_train, y_test = train_test_split(data_x,labels_y,test_size=0.3)

In [9]:
# Logistic Regression has the log linear model
logReg = LogisticRegression()
#Train the model on train data
logReg.fit(train_X, y_train)
#Create list for storing the predictions
predictions = []
for i in test_X:
#predict all the test data
    predictions.append(logReg.predict(i.reshape(1,-1)))
#call the defined metrics function to get the Precison recall and F-measure score
print("Precison, Recall and F-Measure Score for Logistic Regression is:")
print(calculate_metrics(y_test, predictions))

Precison, Recall and F-Measure Score for Logistic Regression is:
(0.6492248062015504, 0.5982142857142857, 0.6226765799256505)


In [10]:
#Maximum Lenght of the Output Dimension
max_Lenght = 128
#Get the data for Test and Train split
dataXnn = data['Tweet_text'].tolist()
labelsYnn = data['Label'].tolist()
#Split the data in to test and train with 70% for train and 30% for test.
trainX_nn, testX_nn, trainY_nn, testY_nn = train_test_split(dataXnn,labelsYnn,test_size=0.3, shuffle=True)
print(len(trainX_nn), len(testX_nn))

2671 1146


In [11]:
#Preprocess data for Keras Model
#Convert class vector to binary class matrix.
trainY_nn = to_categorical(trainY_nn)
#Convert sentences into tokens as an input for the model
tokenizer = Tokenizer(num_words = vocabulary_len)
#Vectorize the corpus
#Update internal vocabulary based on a list of texts
tokenizer.fit_on_texts(trainX_nn)
#Store Label test data before preprocessing into another list so that it can be used later with original format
testY_nn1 = testY_nn
#Transform each text in texts to a sequence of integers
trainX_nn = tokenizer.texts_to_sequences(trainX_nn)
#Padding sequences to the same length. The smaller sententces will be padded with 0 and thus model understands it
trainX_nn = pad_sequences(trainX_nn, maxlen = max_Lenght)
#Transform each text in texts to a sequence of integers for test data
testX_nn = tokenizer.texts_to_sequences(testX_nn)
#Padding sequences to the same length for test data
testX_nn = pad_sequences(testX_nn, maxlen = max_Lenght)
testY_nn = to_categorical(testY_nn)

In [12]:
#Keras Model
#Sequential Model
#Embedd layer for similarity in words
#This layer encodes the input sequence into a sequence of dense vectors.
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_len, max_Lenght), ])
#Implement LSTM layer
#LSTM layer uses gates to control the memorizing process #transforms the vector sequence into a single vector of size comtaining sequence information.
model.add(tf.keras.layers.LSTM(100))
#Dense layer with activation function as softmax
model.add(tf.keras.layers.Dense(2, activation='softmax'))
#Compile the model with binary crossentropy loss functions and stochastic gradient descent optimizer
model.compile(loss = 'binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])
#Summerize the model
model.summary()

2022-08-27 18:50:35.267561: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2182656   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               91600     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 2,274,458
Trainable params: 2,274,458
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Train the model with 10 batch size and 10 iterations
model.fit(x =trainX_nn, y=trainY_nn ,epochs = 10, batch_size = 10,
    validation_data=(testX_nn, testY_nn) )

Epoch 1/10


2022-08-27 18:50:35.763105: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


268/268 [==============================] - 38s 134ms/step - loss: 0.6932 - accuracy: 0.4931 - val_loss: 0.6938 - val_accuracy: 0.4686
Epoch 2/10
268/268 [==============================] - 35s 132ms/step - loss: 0.6930 - accuracy: 0.5107 - val_loss: 0.6938 - val_accuracy: 0.4686
Epoch 3/10
268/268 [==============================] - 35s 131ms/step - loss: 0.6926 - accuracy: 0.5107 - val_loss: 0.6935 - val_accuracy: 0.4703
Epoch 4/10
268/268 [==============================] - 35s 131ms/step - loss: 0.6925 - accuracy: 0.5118 - val_loss: 0.6938 - val_accuracy: 0.4686
Epoch 5/10
268/268 [==============================] - 35s 131ms/step - loss: 0.6922 - accuracy: 0.5241 - val_loss: 0.6939 - val_accuracy: 0.4686
Epoch 6/10
268/268 [==============================] - 35s 129ms/step - loss: 0.6920 - accuracy: 0.5099 - val_loss: 0.6933 - val_accuracy: 0.4738
Epoch 7/10
268/268 [==============================] - 35s 130ms/step - loss: 0.6918 - accuracy: 0.5241 - val_loss: 0.6937 - val_accuracy: 0.4

In [14]:
#Saving the Model
model.save('Final_Model.h5')

In [15]:
#Predicting the classes for the test data and storing in the list
predictRNN = np.argmax(model.predict(testX_nn),axis =1)
#Fecth the Metric results
print("Precison, Recall and F-Measure Score is:")
print(calculate_metrics(testY_nn1,predictRNN))

Precison, Recall and F-Measure Score is:
(0.49096385542168675, 0.9106145251396648, 0.6379647749510763)


# Improved Model:

In [16]:
#Data Enhancement
data = pd.read_csv('../input/tweettxt/tweet.txt', sep='\t')
data.columns = ['Tweet_index', 'Label','Tweet_text']
#Data preprocess and remove the unwanted data
#Removing Numbers
data['Tweet_text'] = data['Tweet_text'].str.replace('\d+', '') #Removing -- character
data['Tweet_text'] = data['Tweet_text'].str.replace("--", '') #Removing Punctuations
data['Tweet_text'] = data['Tweet_text'].str.replace('[^\w\s]','')
#Lowercase and splitting sentences into tokens
data['Tweet_text'] = data['Tweet_text'].str.lower()
#Removing Stop words
data['Tweet_text'].apply(lambda x: [item for item in x if item not in stop])
corpusTweet = data['Tweet_text'].tolist()
wordList =[]
for words in corpusTweet:
      #print(words)
    wordList.extend(words.split())
#wordList = list(itertools.chain.from_iterable(wordList))
vocabulary = set(wordList)
vocabulary_len = len(vocabulary)
#After preprocessing the data, the updating vacabluary has less words
print("Size of the Updating Vocabulary:",vocabulary_len)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


Size of the Updating Vocabulary: 12940


In [17]:
data_x_nn = data['Tweet_text'].tolist()
labels_y = data['Label'].tolist()
#Maximum Lenght of the Output Dimension
max_Lenght = 128
#Get the data for Test Train split
data_x_nn = data['Tweet_text'].tolist()
labels_y_nn = data['Label'].tolist()
#Split the data in to test and train with 70% for train and 30% for test.
trainX_nn, testX_nn, trainY_nn, testY_nn = train_test_split(data_x_nn,labels_y_nn,test_size=0.3, shuffle=True)
print(len(trainX_nn), len(testX_nn))
#Preproces data for Keras Model
#Convert class vector to binary class matrix.
trainY_nn = to_categorical(trainY_nn)
#Converting sentences into tokens as an input for the model
tokenizer = Tokenizer(num_words = vocabulary_len)
#Vectorize the corpus
#Update internal vocabulary based on a list of texts
tokenizer.fit_on_texts(trainX_nn)
#Store Label test data before preprocessing into another list so that it can be used later with original format

2671 1146


In [18]:
testY_nn1 = testY_nn
#Transform each text in texts to a sequence of integers
trainX_nn = tokenizer.texts_to_sequences(trainX_nn)
#Padding sequences to the same length. The smaller sententces will be padded with 0 and thus model understands it
trainX_nn = pad_sequences(trainX_nn, maxlen = max_Lenght)
#Transform each text in texts to a sequence of integers for test data
testX_nn = tokenizer.texts_to_sequences(testX_nn)
#Padding sequences to the same length for test data
testX_nn = pad_sequences(testX_nn, maxlen = max_Lenght)
testY_nn = to_categorical(testY_nn)

In [19]:
#Sequential Model with Embedding layer just like above
model = tf.keras.Sequential([ tf.keras.layers.Embedding(vocabulary_len, max_Lenght),
])
#Add two LSTM layers one returns the sequence other doesnt
model.add(tf.keras.layers.LSTM(100, return_sequences=True))
model.add(tf.keras.layers.LSTM(100, return_sequences=False)) #Drouput layer is used to avoid overfitting. Randomly ignores the nodes from the layer and thus overfitting is avoided
model.add(tf.keras.layers.Dropout(0.4))
#Activation function used is softmax
model.add(tf.keras.layers.Dense(2, activation='relu'))
#Use Hinge loss as it is better for binary classification (From lecture 3 slides) & Adaptive Adam optimizer
model.compile(loss = 'squared_hinge', optimizer=tf.keras.optimizers.Adam(1e-4), metrics = ['accuracy'])
#Summary of Model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1656320   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         91600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,828,522
Trainable params: 1,828,522
Non-trainable params: 0
_________________________________________________________________


In [20]:
#Model Train
model.fit(x =trainX_nn, y=trainY_nn ,epochs = 10, batch_size = 10,
    validation_data=(testX_nn, testY_nn) )

Epoch 1/10
268/268 [==============================] - 75s 264ms/step - loss: 1.0000 - accuracy: 0.4994 - val_loss: 1.0000 - val_accuracy: 0.5244
Epoch 2/10
268/268 [==============================] - 70s 260ms/step - loss: 0.9973 - accuracy: 0.5335 - val_loss: 0.9982 - val_accuracy: 0.4904
Epoch 3/10
268/268 [==============================] - 69s 259ms/step - loss: 0.9589 - accuracy: 0.6189 - val_loss: 0.9863 - val_accuracy: 0.5558
Epoch 4/10
268/268 [==============================] - 71s 264ms/step - loss: 0.7748 - accuracy: 0.8233 - val_loss: 1.0239 - val_accuracy: 0.6274
Epoch 5/10
268/268 [==============================] - 70s 262ms/step - loss: 0.6349 - accuracy: 0.9176 - val_loss: 1.0637 - val_accuracy: 0.5969
Epoch 6/10
268/268 [==============================] - 69s 258ms/step - loss: 0.5662 - accuracy: 0.9659 - val_loss: 1.2104 - val_accuracy: 0.5951
Epoch 7/10
268/268 [==============================] - 70s 260ms/step - loss: 0.5362 - accuracy: 0.9783 - val_loss: 1.2762 - val_ac

In [21]:
ENpredictRNN =model.predict(testX_nn)
classes_x=np.argmax(ENpredictRNN,axis=1)
print(ENpredictRNN)

[[0.1459624  0.37634978]
 [0.50435996 0.04662997]
 [3.1004777  0.        ]
 ...
 [1.241954   0.        ]
 [0.         2.0187237 ]
 [0.         0.8379101 ]]
